In [2]:
import requests

In [3]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
toronto_neighborhood_page = requests.get(wikipedia_link)

In [5]:
page = toronto_neighborhood_page.text

## Using HTMLParser library and adding custom functions

In [6]:
from html.parser import HTMLParser
from html.entities import name2codepoint


class MyHTMLParser(HTMLParser):
    _data = []
    _attr = ('xxx', 'zzzzz')
    _loc_url = []
    _postal = []
    _borough = []
    _neighborhood = []
    _na = []
    _b = 0
    _n = 0
    _l = 'e'
    _le = 'n' 
    m_dup = 0
    _open = 0
    
    def handle_starttag(self, tag, attrs):
        #print( "Start tag:", tag)
        #self._le = self._l
        for attr in attrs:
            if attr[1].find('/wiki/') != -1:
                #print('ATTR[1] WIKI DETECTED')
                self._attr = attr
            #print("     attr:", attr, ' type :', type(attr))

    def handle_endtag(self, tag):
        self._attr = ('xxx', 'zzzzz')
        #print ("End tag  :", tag)

    def handle_data(self, data):
        #self._data.append(data)
        #print('B : ', self._b, ' DATAM b4 :', data )
       
        if data.find('M') != -1 and len(data) == 3:
           #print('DATAM DETECTED')
            self._le = data
            #print(' SLEF._LE', self._le)
            if len(self._postal) !=0:
                if data != self._postal[len(self._postal) - 1]:                
                    self.m_dup = 0     
                    #print('BOROUGH COMPARE B4 : ', self._borough[len(self._borough) - 1], ' DATA: ', data)
                else:
                    self.m_dup = 1
                    self._b = 1
                    #print('BOROUGH COMPARE : ', self._borough[len(self._borough) - 1], ' DATA: ', data)
                    if self._borough[len(self._borough) - 1] != data:
                        self._b = 1
                        
                
            
        if self._b == 1 and data.find('Not assigned') != -1:
            self._neighborhood.append(self._borough[len(self._borough) - 1])
            self._b = 0
        if self._b == 1 and data.find('Not assigned') == -1 and len(data) > 3 and self._attr[1].find('zzzz') != -1:
            #print('UNLINKED DATA DETECTED', data, ' test self._attr[1]: ', self._attr[1])
            if self.m_dup != 0:
                self._neighborhood[len(self._neighborhood) - 1] = self._neighborhood[len(self._neighborhood) - 1] + "," + data
                self._b = 0
            else:  
                self._neighborhood.append(data)
                self._b = 0                    
                
        if data == 'North York':
            self._open = 1
            
        if self._attr[1].find('/wiki/') != -1 and self._open==1:
            #print(' M B4 LOGIC : ', self.m_dup)
            if self._b == 0 and self.m_dup == 0:
                #print('B = 0 DETECTED', " M = 0", ' le :', self._le)
                self._postal.append(self._le)
                self._borough.append(data)
                self._b = 1    
            elif self._b == 1 and self.m_dup == 0:
                #print('B = 1 DETECTED')
                self._neighborhood.append(data)
                self._b = 0
            elif self._b == 1 and self.m_dup == 1:
                if data != self._borough[len(self._borough) - 1]:
                    #print('DUPLICATE IDENTIFIED NEIGH: ', data,  ' b or n: ', self._borough[len(self._borough) - 1] )
                    self._neighborhood[len(self._neighborhood) - 1] = self._neighborhood[len(self._neighborhood) - 1] + "," + data
                    self._b = 0
                    
                
            
        if data == 'South of Bloor':
            self._open = 0   

parser = MyHTMLParser()
parser.feed(page)

### Populate DataFramae using the data optained from the parser

In [7]:
import pandas as pd
t_columns = ['PostalCode','Borough','Neighborhood']
n_df = pd.DataFrame(columns=t_columns)
n_df[t_columns[0]] = parser._postal
n_df[t_columns[1]] = parser._borough
n_df[t_columns[2]] = parser._neighborhood
n_df.head(101)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North\n
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District\n"


In [8]:
n_df.shape

(101, 3)

In [17]:
from geopy.geocoders import Nominatim


address = 'Etobicoke  , Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
lat = location.latitude
lng = location.longitude
print( 'Borough : ',' - ',lat, ' : ', lng)

Borough :   -  43.6435559  :  -79.5656326


### Trying to populate the location using geocoders library. It produces inconsistent results

In [18]:
def find_location():
    lat = []
    lng = []
    t_borough = []
    ex_borough = []
    t_pc = []
    ex_pc = []
    t_neighborhood = []
    
    for i in range(0, len(parser._borough)):
        try:
            address = '{},Toronto'.format(parser._borough[i])
            geolocator = Nominatim(user_agent="toronto_explorer")
            location = geolocator.geocode(address)
            lat.append(location.latitude)
            lng.append(location.longitude)
            t_borough.append(parser._borough[i])
            t_pc.append(parser._postal[i])
            t_neighborhood.append(parser._neighborhood[i])
            #print(i,'The geograpical coordinate of {} are {}, {}.'.format(parser._borough[i],location.latitude, location.longitude))
        
        except:
            ex_borough.append(parser._borough[i])
            ex_pc.append(parser._postal[i])
            #print(i,' EXCEPTION CAUGHT : ', parser._borough[i])
        
    return lat, lng, t_borough,t_neighborhood, t_pc, ex_borough, ex_pc

In [19]:
t_lat, t_lng, t_borough,t_neighborhood,t_pc, ex_borough, ex_pc = find_location()

In [20]:
t_columns = ['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']
toronto_data = pd.DataFrame(columns= t_columns)
toronto_data[t_columns[0]] = t_pc
toronto_data[t_columns[1]] = t_borough
toronto_data[t_columns[2]] = t_neighborhood
toronto_data[t_columns[3]] = t_lat
toronto_data[t_columns[4]] = t_lng

toronto_data.head(98)           


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.770817,-79.413300
1,M4A,North York,Victoria Village,43.770817,-79.413300
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654174,-79.380812
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.770817,-79.413300
4,M7A,Queen's Park,Queen's Park,43.659980,-79.390369
5,M9A,Etobicoke,Islington Avenue,43.643556,-79.565633
6,M1B,Scarborough,"Rouge,Malvern",43.773077,-79.257774
7,M3B,North York,Don Mills North\n,43.770817,-79.413300
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.691339,-79.327821
9,M5B,Downtown Toronto,"Ryerson,Garden District\n",43.654174,-79.380812


### Importing location data from local csv file.

In [21]:
toronto_data_local = pd.read_csv('Dataset/Geospatial_Coordinates.csv')
toronto_data_local.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
pc_local = toronto_data_local['Postal Code'].values
lat_local = toronto_data_local['Latitude'].values
lng_local = toronto_data_local['Longitude'].values

In [23]:
toronto_data_dict = {}
for i in range(0, len(pc_local)):
    toronto_data_dict[pc_local[i]] = [lat_local[i], lng_local[i]]

               
for ind, key in enumerate(toronto_data_dict):
    print(ind, ' : ',key,' :', toronto_data_dict[key])

0  :  M1B  : [43.806686299999996, -79.19435340000001]
1  :  M1C  : [43.7845351, -79.16049709999999]
2  :  M1E  : [43.7635726, -79.1887115]
3  :  M1G  : [43.7709921, -79.21691740000001]
4  :  M1H  : [43.773136, -79.23947609999999]
5  :  M1J  : [43.7447342, -79.23947609999999]
6  :  M1K  : [43.7279292, -79.26202940000002]
7  :  M1L  : [43.711111700000004, -79.2845772]
8  :  M1M  : [43.716316, -79.23947609999999]
9  :  M1N  : [43.692657000000004, -79.2648481]
10  :  M1P  : [43.7574096, -79.27330400000001]
11  :  M1R  : [43.750071500000004, -79.2958491]
12  :  M1S  : [43.7942003, -79.26202940000002]
13  :  M1T  : [43.7816375, -79.3043021]
14  :  M1V  : [43.8152522, -79.2845772]
15  :  M1W  : [43.799525200000005, -79.3183887]
16  :  M1X  : [43.836124700000006, -79.20563609999999]
17  :  M2H  : [43.8037622, -79.3634517]
18  :  M2J  : [43.7785175, -79.3465557]
19  :  M2K  : [43.7869473, -79.385975]
20  :  M2L  : [43.7574902, -79.37471409999999]
21  :  M2M  : [43.789053, -79.40849279999999]
22

### Populating the existing postal data with the imported file.

In [26]:
def final_data():
    final_lat = []
    final_lng = []
    final_pc = []
    final_bor = []
    final_nei = []

    for ind, key in enumerate(parser._postal):
        final_lat.append(toronto_data_dict[key][0])
        final_lng.append(toronto_data_dict[key][1])
        final_pc.append(parser._postal[ind])
        final_bor.append(parser._borough[ind])
        final_nei.append(parser._neighborhood[ind])
        
    return final_pc, final_bor, final_nei, final_lat, final_lng

In [27]:
final_pc, final_bor, final_nei, final_lat, final_lng = final_data()

In [28]:
f_toronto_data = pd.DataFrame(columns = t_columns)
f_toronto_data[t_columns[0]] = final_pc
f_toronto_data[t_columns[1]] = final_bor
f_toronto_data[t_columns[2]] = final_nei
f_toronto_data[t_columns[3]] = final_lat
f_toronto_data[t_columns[4]] = final_lng

In [29]:
f_toronto_data.shape

(101, 5)

In [30]:
f_toronto_data.head(101)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North\n,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District\n",43.657162,-79.378937
